In [1]:
import string
import numpy as np
import PIL.Image
from os import listdir
from pickle import dump, load
from numpy import array
from numpy import argmax
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout
from keras.callbacks import ModelCheckpoint
from nltk.translate.bleu_score import corpus_bleu

Using TensorFlow backend.


# Preparing Photo Data

### About VGG16 model:


Application:

1. Given image → find object name in the image.

2. It can detect any one of 1000 images.

3. It takes input image of size 224 * 244 * 3 (RGB image).


Built using:

1. Convolutions layers (used only 3*3 size ).

2. Max pooling layers (used only 2*2 size).

3. Fully connected layers at end.

4. Total 16 layers.

In [2]:
def extract_photo_features(directory):
    #preparing VGG16 model
    model = VGG16()
    #removing the last layer from the model since we require only the features and not the classification
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    #summarizing the model
    print(model.summary())
    #storing the features in a dictionary
    photo_features = dict()
    for name in listdir(directory):
        image_name = directory + '/' + name
        #loading the image and converting it to 224 X 224 size
        image = load_img(image_name, target_size=(224,224))
        #converting image to a 3D numpy array
        image = img_to_array(image)
        #reshaping data for the model
        image = image.reshape(1, image.shape[0], image.shape[1], image.shape[2])
        #preprocessing the images for the model
        #The preprocess_input function is meant to adequate your image to the format the model requires.
        #Some models use images with values ranging from 0 to 1. Others from -1 to +1.
        #Others use the "caffe" style, that is not normalized, but is centered.
        image = preprocess_input(image)
        #getting feature of a single image
        photo_feature = model.predict(image, verbose=0)
        #getting the image name
        image_id = name.split('.')[0]
        #storing the feature corresponding to the image in the dictionary
        photo_features[image_id] = photo_feature
        
        print(name)
    return photo_features

In [3]:
#the directory we are using
directory = 'Flicker8k_Dataset'
#calling the function
features = extract_photo_features(directory)
print('Extracted Features: ', len(features))

#dumping the features in a pickle file for further use
dump(features, open('features.pkl', 'wb'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

KeyboardInterrupt: 

# Preparing Text Data

In [3]:
#function to load the file containing all the description
def load_description_file(filename):
    #opening the description file in read mode
    file = open(filename, 'r')
    #reading the file and storing the descriptions
    descriptions = file.read()
    #closing the file
    file.close()
    
    return descriptions

#function to return a dictionary of photo identifiers to the descriptions
def photo_to_description_mapping(descriptions):
    #dictionary to store the mapping of photo identifiers to descriptions
    description_mapping = dict()
    #iterating through each line of the descriptions
    for line in descriptions.split('\n'):
        #splitting the lines by white space
        words = line.split()
        #skipping the lines with length less than 2
        if len(line)<2:
            continue
        #the first word is the image_id and the rest are the part of the description of that image
        image_id, image_description = words[0], words[1:]
        #retaining only the name of the image and removing the extension from it
        image_id = image_id.split('.')[0]
        #image_descriptions contains comma separated words of the description, hence, converting it back to string
        image_description = ' '.join(image_description)
        #there are multiple descriptions per image, hence, corresponding to every image identifier in the
        #dictionary, there is a list of description
        #if the list does not exist then we need to create it
        if image_id not in description_mapping:
            description_mapping[image_id] = list()
        #now storing the descriptions in the mapping
        description_mapping[image_id].append(image_description)
    
    return description_mapping

#function to clean the descriptions in the following ways:
#     Convert all words to lowercase.
#     Remove all punctuation.
#     Remove all words that are one character or less in length (e.g. ‘a’).
#     Remove all words with numbers in them.
def clean_descriptions(description_mapping):
    #making a translation table for removing all the punctuation
    table = str.maketrans('','', string.punctuation)
    #traversing through the mapping we created
    for key, descriptions in description_mapping.items():
        for i in range(len(descriptions)):
            description = descriptions[i]
            description = description.split()
            #converting all the words to lower case
            description = [word.lower() for word in description]
            #removing the punctuation using the translation table we made
            description = [word.translate(table) for word in description]
            #removing the words with length =1
            description = [word for word in description if len(word)>1]
            #removing all words with number in them
            description = [word for word in description if word.isalpha()]
            #converting the description back to string and overwriting in the descriptions list
            descriptions[i] = ' '.join(description)

#converting the loaded descriptions into a vocabulary of words
# Ideally, we want a vocabulary that is both expressive and as small as possible.
# A smaller vocabulary will result in a smaller model that will train faster.
# For reference, we can transform the clean descriptions into a set 
# and print its size to get an idea of the size of our dataset vocabulary.
def vocabulary(descriptions):
    #creating a set of all descriptions
    #sets are highly optimized, don't contain any duplicate values
    #there implementation is based on hash table
    #hence we get a vocabulary that is both expressive and small
    all_descriptions = set()
    for key in descriptions.keys():
        [all_descriptions.update(desc.split()) for desc in descriptions[key]]
    
    return all_descriptions

#function to save the cleaned descriptions into a file one per line
def save_cleaned_descriptions(cleaned_descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + ' ' + desc)
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [4]:
filename = 'Flickr8k_text/Flickr8k.token.txt'
description_file = load_description_file(filename)
descriptions = photo_to_description_mapping(description_file)
print('Loaded descriptions: ', len(descriptions))
clean_descriptions(descriptions)
vocabulary = vocabulary(descriptions)
print('Vocabulary Size: ', len(vocabulary))
save_cleaned_descriptions(descriptions, 'descriptions.txt')

Loaded descriptions:  8092
Vocabulary Size:  8763


# Preparing Deep Learning Model

## Loading Data

In [5]:
#general function for loading a file into memeory and return text from it
def load_file(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

#function for loading a predefined list of photo identifiers
def load_photo_identifiers(filename):
    #loading the file containing the list of photo identifier
    file = load_file(filename)
    #creating a list for stiring the identifiers
    photos = list()
    #traversing the file one line at a time
    for line in file.split('\n'):
        if len(line) < 1:
            continue
        #image name contains the extension as well. but we just the name
        identifier = line.split('.')[0]
        #adding it to the list of photos
        photos.append(identifier)
    #returning the set of photos created
    return set(photos)


#loading the cleaned descriptions that we created earlier
#we will only be loading the descriptions of the images that we will use for training
#hence we need to pass the set of train photos that the above function will be returning
def load_clean_descriptions(filename, photos):
    #loading the cleaned description file
    file = load_file(filename)
    #creating a dictionary of descripitions for storing the photo to description mapping of train images
    descriptions = dict()
    #traversing the file line by line
    for line in file.split('\n'):
        #splitting the line at white spaces
        words = line.split()
        #the first word will be the image name and the rest will be the description of that particular image
        image_id, image_description = words[0], words[1:]
        #we want to load only those description which corresponds to the set of photos we provided as argument
        if image_id in photos:
            #creating list of description if needed
            if image_id not in descriptions:
                descriptions[image_id] = list()
            #the model we will develop will generate a caption given a photo, 
            #and the caption will be generated one word at a time. 
            #The sequence of previously generated words will be provided as input. 
            #Therefore, we will need a ‘first word’ to kick-off the generation process 
            #and a ‘last word‘ to signal the end of the caption.
            #we will use 'startseq' and 'endseq' for this purpose
            #also we have to conver image description back to string
            desc = 'startseq ' + ' '.join(image_description) + ' endseq'
            descriptions[image_id].append(desc)
            
    return descriptions

#function to load the photo features created using the VGG16 model
def load_photo_features(filename, photos):
    #this will load the entire features
    all_features = load(open(filename, 'rb'))
    #we are interested in loading the features of the required photos only
    features = {k: all_features[k] for k in photos}
    return features

In [6]:
filename = 'Flickr8k_text/Flickr_8k.trainImages.txt'
train = load_photo_identifiers(filename)
print('Dataset: ',len(train))
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=', len(train_descriptions))
train_features = load_photo_features('features.pkl', train)
print('Photos: train=', len(train_features))

Dataset:  6000
Descriptions: train= 6000
Photos: train= 6000


### The description text will need to be encoded to numbers before it can be presented to the model as in input or compared to the model’s predictions.

### The first step in encoding the data is to create a consistent mapping from words to unique integer values. Keras provides the Tokenizer class that can learn this mapping from the loaded description data.

### Below defines the to_lines() to convert the dictionary of descriptions into a list of strings and the create_tokenizer() function that will fit a Tokenizer given the loaded photo description text.

In [7]:
#convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
    all_desc = list()
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

#given the descriptions, fit a tokenizer

#TOKENIZER CLASS:
#This class allows to vectorize a text corpus, 
#by turning each text into either a sequence of integers 
#(each integer being the index of a token in a dictionary) 
#or into a vector where the coefficient for each token could be binary, based on word count, based on tf-idf...
def create_tokenizer(descriptions):
    lines = to_lines(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [8]:
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: ', vocab_size)

Vocabulary Size:  7579


### We can now encode the text.

Each description will be split into words. The model will be provided one word and the photo and generate the next word. Then the first two words of the description will be provided to the model as input with the image to generate the next word. This is how the model will be trained.

In [9]:
#the function below, given the tokenizer, max length of sequence, dictionary of descriptions and photo features,
#will transform data in input-output pairs of data for training the model. There are two input arrays to the model,
#one for the encoded text and one for the photo features. There is one output for the model i.e.
#the encoded next word
#The output will therefore be a one hot encoded version of each word reperesenting an idealized probability
#distribution with 0 value at all word positions except the actual word position which is represented by 1.
# def create_sequence(tokenizer, max_length, descriptions, photos):
#     x1, x2, y = list(), list(), list()
#     for key, description_list in descriptions.items():
#         for description in description_list:
#             sequence = tokenizer.text_to_sequences([description])[0]
#             for i in range(1, len(sequence)):
#                 input_sequence, output_sequence = sequence[:i], sequence[i]
#                 #pad_sequences function transforms a list of num_samples sequences (lists of integers) into a 2D Numpy array
#                 #of shape (num_samples, num_timesteps). num_timesteps is either the maxlen argument if provided, 
#                 #or the length of the longest sequence otherwise.
#                 input_sequence = pad_sequences([input_sequence], maxlen=max_length)[0]
#                 #to_categorical function transforms a class vector (integers) to binary class matrix.
#                 output_sequence = to_categorical([output_sequence], num_classes=vocab_size)[0]
#                 x1.append(photos[key][0])
#                 x2.append(input_sequence)
#                 y.append(output_sequence)
#     return array(x1), array(x2), array(y)

#calculated the length of description with most words
def max_length(descriptions):
    lines = to_lines(descriptions)
    return max(len(d.split()) for d in lines)

# Defining Model

### The model is in three parts:

1. Photo Feature Extractor. This is a 16-layer VGG model pre-trained on the ImageNet dataset. We have pre-processed the photos with the VGG model (without the output layer) and will use the extracted features predicted by this model as input.
2. Sequence Processor. This is a word embedding layer for handling the text input, followed by a Long Short-Term Memory (LSTM) recurrent neural network layer.
3. Decoder. Both the feature extractor and sequence processor output a fixed-length vector. These are merged together and processed by a Dense layer to make a final prediction.

The Photo Feature Extractor model expects input photo features to be a vector of 4,096 elements. These are processed by a Dense layer to produce a 256 element representation of the photo.

The Sequence Processor model expects input sequences with a pre-defined length (34 words) which are fed into an Embedding layer that uses a mask to ignore padded values. This is followed by an LSTM layer with 256 memory units.

Both the input models produce a 256 element vector. Further, both input models use regularization in the form of 50% dropout. This is to reduce overfitting the training dataset, as this model configuration learns very fast.

The Decoder model merges the vectors from both input models using an addition operation. This is then fed to a Dense 256 neuron layer and then to a final output Dense layer that makes a softmax prediction over the entire output vocabulary for the next word in the sequence.

## Training with progressive loading

Progressive Loading
The Flicr8K dataset of photos and descriptions can fit into RAM, if you have a lot of RAM (e.g. 8 Gigabytes or more), and most modern systems do.

This is fine if you want to fit a deep learning model using the CPU.

Alternately, if you want to fit a model using a GPU, then you will not be able to fit the data into memory of an average GPU video card.

One solution is to progressively load the photos and descriptions as-needed by the model.

Keras supports progressively loaded datasets by using the fit_generator() function on the model. A generator is the term used to describe a function used to return batches of samples for the model to train on. This can be as simple as a standalone function, the name of which is passed to the fit_generator() function when fitting the model.

As a reminder, a model is fit for multiple epochs, where one epoch is one pass through the entire training dataset, such as all photos. One epoch is comprised of multiple batches of examples where the model weights are updated at the end of each batch.

A generator must create and yield one batch of examples. For example, the average sentence length in the dataset is 11 words; that means that each photo will result in 11 examples for fitting the model and two photos will result in about 22 examples on average. A good default batch size for modern hardware may be 32 examples, so that is about 2-3 photos worth of examples.

We can write a custom generator to load a few photos and return the samples as a single batch.

Let’s assume we are working with a word-by-word model described in the previous section that expects a sequence of words and a prepared image as input and predicts a single word.

Let’s design a data generator that given a loaded dictionary of image identifiers to clean descriptions, a trained tokenizer, and a maximum sequence length will load one-image worth of examples for each batch.

In [10]:
#the below function loop forever with a while loop and within this, 
#loop over each image in the image directory. 
#For each image filename, we can load the image and 
#create all of the input-output sequence pairs from the image’s description.

#data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, tokenizer, max_length):
    while 1:
        for key, description_list in descriptions.items():
            #retrieve photo features
            photo = photos[key][0]
            input_image, input_sequence, output_word = create_sequence(tokenizer, max_length, description_lsit, photo)
            yield [[input_image, input_sequence], output_word]

            
#we are calling the create_sequence() function to create 
#a batch worth of data for a single photo rather than an entire dataset. 
#This means that we must update the create_sequences() function 
#to delete the “iterate over all descriptions” for-loop.            
#Updated create sequence function for data_generator
def create_sequences(tokenizer, max_length, desc_list, photo):
    X1, X2, y = list(), list(), list()
    # walk through each description for the image
    for desc in desc_list:
        # encode the sequence
        sequence = tokenizer.texts_to_sequences([desc])[0]
        # split one sequence into multiple X,y pairs
        for i in range(1, len(sequence)):
            input_sequence, output_sequence = sequence[:i], sequence[i]
            #pad_sequences function transforms a list of num_samples sequences (lists of integers) into a 2D Numpy array
            #of shape (num_samples, num_timesteps). num_timesteps is either the maxlen argument if provided, 
            #or the length of the longest sequence otherwise.
            input_sequence = pad_sequences([input_sequence], maxlen=max_length)[0]
            #to_categorical function transforms a class vector (integers) to binary class matrix.
            output_sequence = to_categorical([output_sequence], num_classes=vocab_size)[0]
            x1.append(photo)
            x2.append(in_seq)
            y.append(out_seq)
    return array(x1), array(x2), array(y)

In [11]:
def define_model(vocab_size, max_length):
    # feature extractor model
    inputs1 = Input(shape=(4096,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
    # sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)
    # decoder model
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    # compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    # summarize model
    model.summary()
    return model

In [27]:
filename = 'Flickr8k_text/Flickr_8k.trainImages.txt'
train = load_photo_identifiers(filename)
print('Dataset: ', len(train))
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=', len(train_descriptions))
train_features = load_photo_features('features.pkl', train)
print('Photos: train=', len(train_features))
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size:', vocab_size)
max_length = max_length(train_descriptions)
print('Description Length: ', max_length)

model = define_model(vocab_size, max_length)
epochs = 20
steps = len(train_descriptions)
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('model_' + str(i) + '.h5')

Dataset:  6000
Descriptions: train= 6000
Photos: train= 6000
Vocabulary Size: 7579
Description Length:  34
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 34)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 4096)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 34, 256)      1940224     input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 4096)         0           input_1[0][0]                    
__

NameError: name 'description_lsit' is not defined

# Evaluation

In [10]:
#this function maps an integer to a word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

#The function below generates a textual description given a trained model, 
#and a given prepared photo as input. It calls the function word_for_id() 
#in order to map an integer prediction back to a word.
def generate_desc(model, tokenizer, photo, max_length):
    #start tge generation process
    in_text = 'startseq'
    #iterating over the max_length since the maximum length of the description can be that only
    for i in range(max_length):
        #integer ncoding input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        #padding the input
        sequence = pad_sequences([sequence], maxlen=max_length)
        #predicting next word
        #the predict function will return probability
        prob = model.predict([photo,sequence], verbose=0)
        #converting the probability to integer
        prob = argmax(prob)
        #calling the word_for_id function in order to map integer to word
        word = word_for_id(prob, tokenizer)
        #breaking if word cannot be mapped
        if word is None:
            break
        #appending as input
        in_text += ' ' + word
        #break if end is predicted
        if word == 'endseq':
            break
    return in_text

#the below function evaluates the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
    actual, predicted = list(), list()
    for key, desc_list in descriptions.items():
        prediction = generate_desc(model, tokenizer, photos[key], max_length)
        actual_desc = [d.split() for d in desc_list]
        actual.append(actual_desc)
        predicted.append(prediction.split())

    print('BLEU-1: ', corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: ', corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: ', corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: ', corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))
    
def max_length(descriptions):
    lines = to_lines(descriptions)
    return max(len(d.split()) for d in lines)

In [42]:
filename = 'Flickr8k_text/Flickr_8k.trainImages.txt'
train = load_photo_identifiers(filename)
print('Dataset: ', len(train))
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=', len(train_descriptions))
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: ', vocab_size)
max_length = max_length(train_descriptions)
print('Description Length: ,', max_length)

filename = 'Flickr8k_text/Flickr_8k.testImages.txt'
test = load_photo_identifiers(filename)
print('Dataset: ', len(test))
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print('Descriptions: test=', len(test_descriptions))
test_features = load_photo_features('features.pkl', test)
print('Photos: test=', len(test_features))

filename = 'model_12.h5'
model = load_model(filename)
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)

Dataset:  6000
Descriptions: train= 6000
Vocabulary Size:  7579
Description Length: , 34
Dataset:  1000
Descriptions: test= 1000
Photos: test= 1000
BLEU-1:  0.5079996300749098
BLEU-2:  0.26815254395800053
BLEU-3:  0.17788590578166746
BLEU-4:  0.07615229858117227


# Generate New Descriptions

In [12]:
filename = 'Flickr8k_text/Flickr_8k.trainImages.txt'
train = load_photo_identifiers(filename)
print('Dataset: ', len(train))
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=', len(train_descriptions))
tokenizer = create_tokenizer(train_descriptions)
dump(tokenizer, open('tokenizer.pkl', 'wb'))

Dataset:  6000
Descriptions: train= 6000


In [2]:
def extract_features(filename):
    model = VGG16()
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    image = load_img(filename, target_size=(224, 224))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    feature = model.predict(image, verbose=0)
    return feature

def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = argmax(yhat)
        word = word_for_id(yhat, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    return in_text

tokenizer = load(open('tokenizer.pkl', 'rb'))
max_length = 34

In [4]:
model = load_model('model_13.h5')
photo = extract_features('example.jpg')
description = generate_desc(model, tokenizer, photo, max_length)
print(description)

startseq man in black shirt is walking on the street endseq
